In [293]:
import requests
import lxml
import pandas as pd
from bs4 import BeautifulSoup # Html search tool
import re # Regular expressions

In [294]:
# Let's not use context managers for now?
"""
with requests.Session() as main_session:
   city_state_abbr = 'raleigh-nc/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/' + city_state_abbr
   main_request = main_session.get(url, headers=request_headers)

# After finding urls from the main page, use those links to open homedetaills page
with requests.Session() as branch_session:
    branch_base_url = "https://www.zillow.com/homedetails/"
    branch_sub_url = "5904-Endsley-Ct-Raleigh-NC-27610/65332868_zpid/"

    branch_request = branch_session.get(branch_base_url + branch_sub_url, headers=request_headers)

branch_soup = BeautifulSoup(branch_request.content, "html.parser")"""

'\nwith requests.Session() as main_session:\n   city_state_abbr = \'raleigh-nc/\' #*****change this city to what you want*****\n   url = \'https://www.zillow.com/\' + city_state_abbr\n   main_request = main_session.get(url, headers=request_headers)\n\n# After finding urls from the main page, use those links to open homedetaills page\nwith requests.Session() as branch_session:\n    branch_base_url = "https://www.zillow.com/homedetails/"\n    branch_sub_url = "5904-Endsley-Ct-Raleigh-NC-27610/65332868_zpid/"\n\n    branch_request = branch_session.get(branch_base_url + branch_sub_url, headers=request_headers)\n\nbranch_soup = BeautifulSoup(branch_request.content, "html.parser")'

In [295]:
"""#add contents of urls to soup variable from each url
main_request_soup = BeautifulSoup(main_request.content, "html.parser")

# From intial queried page (e.g., homes in Raleigh) find all links to homedetails page,
# which will have more detailed information on the home.

# This will be the main base where the scraper will come back to after scraping each
# home details page. 

# Example: queried page -> homedetails page -> scrape data -> queried page 
# Loop until all homes are scraped, then proceed to next page.
homedetails_urls_list = main_request_soup.find_all(href=re.compile("homedetails"))
# homedetails_urls_list = main_request_soup.select('a[href^="https://www.zillow.com/homedetails/"]')

print(len(homedetails_urls_list))
print(main_request.links)"""


'#add contents of urls to soup variable from each url\nmain_request_soup = BeautifulSoup(main_request.content, "html.parser")\n\n# From intial queried page (e.g., homes in Raleigh) find all links to homedetails page,\n# which will have more detailed information on the home.\n\n# This will be the main base where the scraper will come back to after scraping each\n# home details page. \n\n# Example: queried page -> homedetails page -> scrape data -> queried page \n# Loop until all homes are scraped, then proceed to next page.\nhomedetails_urls_list = main_request_soup.find_all(href=re.compile("homedetails"))\n# homedetails_urls_list = main_request_soup.select(\'a[href^="https://www.zillow.com/homedetails/"]\')\n\nprint(len(homedetails_urls_list))\nprint(main_request.links)'

In [296]:
"""# Header required to circumvent captcha
request_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
city_state_abbr = "raleigh-nc/" #*****change this city to what you want*****
url = "https://www.zillow.com/" + city_state_abbr + page_number + "_p"

main_request = requests.get(url, headers=request_headers)"""

In [312]:
request_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }

In [340]:
def get_all_urls(city_state_abbr="raleigh-nc/"):

    url_setup = "https://www.zillow.com/" + city_state_abbr
    main_request_setup = requests.get(url_setup, headers=request_headers)
    page_soup = BeautifulSoup(main_request_setup.content).find_all("span", class_ = "Text-c11n-8-73-8__sc-aiai24-0 bKahKV")

    page_soup_max_number = int(re.search("Page <!-- -->1<!-- --> of <!-- -->(.+?)</span>", str(page_soup)).group(1))

    full_list_urls = []

    for page_number in range(1, page_soup_max_number + 1):
        url = "https://www.zillow.com/" + city_state_abbr + str(page_number) + "_p"
        main_request = requests.get(url, headers=request_headers)
        main_re_search = list(set(re.findall("https://www.zillow.com/homedetails/(.+?)_zpid", str(main_request.content))))
        full_list_urls.append(main_re_search)

    flat_full_llist_urls = [item for sublist in full_list_urls for item in sublist]

    return flat_full_llist_urls

In [341]:
buncha_urls = get_all_urls()
len(buncha_urls)

782

In [310]:

#main_re_search = set(re.findall("https://www.zillow.com/homedetails/(.+?)_zpid", str(main_request.content)))
#re_search_set = set(main_re_search) # Set creates unique list - could change this in future if needed.
#print(main_re_search)


In [ ]:
def get_full_url_branch(re_search_set):

    full_url_list = []

    for url in re_search_set:
        #print(url)
        full_url = "https://www.zillow.com/homedetails/" + url + "_zpid"
        #print(full_url)
        full_url_list.append(full_url)
    
    return full_url_list

# print(get_full_url_branch(main_re_search))

In [299]:
# 43.3s
def get_branch_request(full_url_list):

    branch_response_good_soup_list = []

    for url in full_url_list:
        branch_response = requests.get(url, headers=request_headers)
        branch_response_good_soup = BeautifulSoup(branch_response.content, "html.parser")
        branch_response_good_soup_list.append(branch_response_good_soup)

    return branch_response_good_soup_list

#print(get_branch_request(get_full_url_branch(main_re_search)))

In [300]:
# Create a function that goes into the homedetails branch url and grabs
# Price, beds, baths, sqft, days on zillow, views, and saves

def scrape_homedetails(branch_homedetails_url_list):
    """
    Scrape the homedetails url for price, beds, baths, 
    sqft, days on zillow, views, and saves
    """

    page_scraped_data_list = []

    for soup in branch_homedetails_url_list:
        # Raw soups to be used below
        raw_price_soup = soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY")
        #raw_bed_soup = soup.find_all("strong")
        raw_address_soup = soup.find("h1")

        # Price
        raw_price_soup = str(soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY"))
        price_match_object = re.search("<span>(.+?)</span>", raw_price_soup)
        price = price_match_object.group(1)

        # Attributes list: 0 Bed, 1 bath,  2 sqft, 3 days on zillow, 5 views, 6 saves
        raw_bed_soup = str(soup.find_all("strong"))
        raw_bed_soup_list = raw_bed_soup.split(', ') # Holy balls spent 4 hours on this.

        
        count_beds = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[0]).group(1)
        count_baths = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[1]).group(1)
        sq_ft = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[2]).group(1)
        days_on_zillow = re.search("<strong>(.+?)", raw_bed_soup_list[3]).group(1)
        count_views = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[4]).group(1)
        count_saves = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[5]).group(1)

        # Done
        address = str(raw_address_soup)
        address_1 = re.search("kHeRng(.+?)<!--", address).group(1)[2:]
        address_2 = re.search("<!-- -->(.+?)</h1>", address).group(1)[9:]
        address_full = address_1 + ' ' + address_2

        # Aggregate into list
        aggregate_list = [address_full, price, count_beds, count_baths, sq_ft, days_on_zillow,
                        count_views, count_saves]

        page_scraped_data_list.append(aggregate_list)
    
    return page_scraped_data_list

In [343]:
page_data_1 = scrape_homedetails(get_branch_request(get_full_url_branch(get_all_urls())))
omg_df = pd.DataFrame(page_data_1)
omg_df.shape

IndexError: list index out of range

In [345]:
omg_df.shape

(40, 8)

For pagination 

only difference is 2_p for page 2 3_p for page 3

https://www.zillow.com/raleigh-nc/luxury-homes/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D

https://www.zillow.com/raleigh-nc/luxury-homes/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D

https://www.zillow.com/raleigh-nc/luxury-homes/3_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Raleigh%2C%20NC%22%2C%22mapBounds%22%3A%7B%22west%22%3A-78.92658305517578%2C%22east%22%3A-78.32439494482422%2C%22south%22%3A35.45233694912461%2C%22north%22%3A36.119629123743984%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54047%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priced%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22pagination%22%3A%7B%22currentPage%22%3A3%7D%7D

Unsure of use of code below this:

In [301]:
test_output_str = str(test_output)
href_url = test_output_str.split(' ')[5]
href_url_strip = href_url.replace("href=","")
href_url_strip = href_url_strip.replace('"','')
print(href_url_strip)

NameError: name 'test_output' is not defined

In [ ]:
# Simply testing function, can remove
test_output = find_all_keyword_units(keyword = "homedetails")
print(test_output)

In [ ]:
# Replaced with regular expression re?
# Find all sections from main_soup that have homedetails. This will be used to get homedetails urls
def find_all_keyword_units(keyword="homedetails"):
    """Returns a list of keyword matches"""
    keyword_match_list = main_request_soup.find_all(href=re.compile(str(keyword)))
    return keyword_match_list

# Replaced with re?
# Create function to take list and return pruned list including only urls for homedetails
def strip_list_to_url(html_list, index=5):
    """Takes keyword soup search, parses it into a list, and selects the homedetails url"""
    list_to_str = str(html_list)
    href_url = list_to_str.split(' ')[index] # Selects index of href with housedetails
    href_url_strip = href_url.replace("href=","")
    href_url_strip = href_url_strip.replace('"','')
    return href_url_strip

In [ ]:
# pretty sure I don't need this...
"""
# https://blog.devgenius.io/scraping-zillow-with-python-and-beautifulsoup-bbc7e581c218

#create the first two dataframes
df = pd.DataFrame()
df1 = pd.DataFrame()
#all for loops are pulling the specified variable using beautiful soup and inserting into said variable
for i in soup:
    address = soup.find_all (class_= 'list-card-addr')
    price = list(soup.find_all (class_='list-card-price'))
    beds = list(soup.find_all("ul", class_="list-card-details"))
    details = soup.find_all ('div', {'class': 'list-card-details'})
    home_type = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link = soup.find_all (class_= 'list-card-link')
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df['links'] = urls
df['links'] = df['links'].astype('str')

#remove html tags
df['links'] = df['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)
"""

'\n# https://blog.devgenius.io/scraping-zillow-with-python-and-beautifulsoup-bbc7e581c218\n\n#create the first two dataframes\ndf = pd.DataFrame()\ndf1 = pd.DataFrame()\n#all for loops are pulling the specified variable using beautiful soup and inserting into said variable\nfor i in soup:\n    address = soup.find_all (class_= \'list-card-addr\')\n    price = list(soup.find_all (class_=\'list-card-price\'))\n    beds = list(soup.find_all("ul", class_="list-card-details"))\n    details = soup.find_all (\'div\', {\'class\': \'list-card-details\'})\n    home_type = soup.find_all (\'div\', {\'class\': \'list-card-footer\'})\n    last_updated = soup.find_all (\'div\', {\'class\': \'list-card-top\'})\n    brokerage = list(soup.find_all(class_= \'list-card-brokerage list-card-img-overlay\',text=True))\n    link = soup.find_all (class_= \'list-card-link\')\n    \n    #create dataframe columns out of variables\n    df[\'prices\'] = price\n    df[\'address\'] = address\n    df[\'beds\'] = beds\n\

In [ ]:
# Appendix:

# Done - address
address = str(raw_address_soup)
address_1 = re.search("kHeRng(.+?)<!--", address).group(1)[2:]
address_2 = re.search("<!-- -->(.+?)</h1>", address).group(1)[9:]


print(address_1)
print(address_2)

# Price
raw_price_soup = str(branch_soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY"))
price_match_object = re.search("<span>(.+?)</span>", raw_price_soup)
price = price_match_object.group(1)

print(raw_price_soup)
print(price)

# Done
# Get all attributes besides price and address
# Turn this into a function bb

# 0 Bed, 1 bath,  2 sqft, 3 days on zillow, 5 views, 6 saves
raw_bed_soup = str(branch_soup.find_all("strong"))
raw_bed_soup_list = raw_bed_soup.split(' ')
count_beds = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[0]).group(1)
count_baths = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[1]).group(1)
sq_ft = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[2]).group(1)
days_on_zillow = re.search("<strong>(.+?)", raw_bed_soup_list[3]).group(1)
count_views = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[5]).group(1)
count_saves = re.search("<strong>(.+?)</strong>", raw_bed_soup_list[6]).group(1)


print("beds",count_beds)
print("baths", count_baths)
print("sq_ft", sq_ft)
print("days_on_zillow", days_on_zillow)
print("count_views", count_views)
print("count_saves", count_saves)

#raw soups
raw_price_soup = branch_response_soup.find_all("span", class_ = "Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY")
raw_bed_soup = branch_response_soup.find_all("strong")
raw_address_soup = branch_response_soup.find("h1")

print(raw_price_soup) # done
print(raw_bed_soup) # done
print(raw_address_soup) # done


2717 Hiking Trl,
Raleigh, NC 27615
[<span class="Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY" data-testid="price"><span>$352,000</span></span>]
$352,000
beds 3
baths 3
sq_ft 1,566
days_on_zillow 8
count_views 3,931
count_saves 229
[<span class="Text-c11n-8-73-0__sc-aiai24-0 dpf__sc-1me8eh6-0 kGdfMs fzJCbY" data-testid="price"><span>$352,000</span></span>]
[<strong>3</strong>, <strong>3</strong>, <strong>1,566</strong>, <strong>8 days</strong>, <strong>3,940</strong>, <strong>229</strong>, <strong>WED</strong>, <strong>THU</strong>, <strong>FRI</strong>, <strong>SAT</strong>]
<h1 class="Text-c11n-8-73-0__sc-aiai24-0 kHeRng">5904 Endsley Ct,<!-- --> <!-- -->Raleigh, NC 27610</h1>
